In [1]:
from tensorflow import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.4.0
2.4.0


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 15

# Florida voter
df = pd.read_csv('../dataverse_files/fl_reg_name_race.csv.gz')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df[df.race.isin(['multi_racial', 'native_indian', 'other', 'unknown']) == False].sample(SAMPLE, random_state=21)
del df

# Additional features
sdf['name_last'] = sdf.name_last.str.title()

sdf

,name_first,name_last,race
841323,Torres,Jose,hispanc
1408926,Da Silva,Amanda,nh_white
1733118,Mc Ghee,Sandra,nh_white
13104513,KARAM,Melinda,nh_white
9156114,BREWER,Laia,nh_black
...,...,...,...
3076722,Antunez Avila,Robert,hispanc
10023679,DAVIS,Wyatt,nh_white
5846252,Scott,Jacquelyn,nh_white
5959131,Parton,Douglas,nh_white


In [3]:
rdf = sdf.groupby('race').agg({'name_last': 'count'})
rdf.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_ln_race.csv', columns=[])
rdf

,name_last
race,
asian,19431
hispanc,166865
nh_black,142675
nh_white,671029


In [4]:
sdf.groupby('race').agg({'name_last': 'nunique'})

,name_last
race,
asian,8933
hispanc,17512
nh_black,33422
nh_white,25575


## Preprocessing the input data

In [5]:
# last name only
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1103
Max feature len = 25, Avg. feature len = 5


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [6]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


In [7]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 32)            35296     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 118,244
Trainable params: 118,244
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15
22500/22500 [==============================] - 413s 18ms/step - loss: 0.7442 - accuracy: 0.7376 - val_loss: 0.6717 - val_accuracy: 0.7645
Epoch 2/15
22500/22500 [==============================] - 401s 18ms/step - loss: 0.6689 - accuracy: 0.7662 - val_loss: 0.6499 - val_accuracy: 0.7744
Epoch 3/15
22500/22500 [==============================] - 392s 17ms/step - loss: 0.6516 - accuracy: 0.7735 - val_loss: 0.6390 - val_accuracy: 0.7796
Epoch 4/15
22500/22500 [==============================] - 391s 17ms/step - loss: 0.6408 - accuracy: 0.7777 - val_loss: 0.6338 - val_accuracy: 0.7826
Epoch 5/15
22500/22500 [==============================] - 391s 17ms/step - loss: 0.6364 - accuracy: 0.7798 - val_loss: 0.6339 - val_accuracy: 0.7820
Epoch 6/15
22500/22500 [==============================] - 391s 17ms/step - loss: 0.6318 - accuracy: 0.7816 - val_loss: 0.6306 - val_accuracy: 0.7832
Epoch 7/15
22500/22500 [==============================] - 390s 17ms/step - loss: 0.6291 - accurac

In [9]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.6254821419715881
Test accuracy: 0.7856850028038025


## Confusion Matrix

In [10]:
p = model.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

6250/6250 - 21s
              precision    recall  f1-score   support

       asian       0.70      0.31      0.42      3886
     hispanc       0.75      0.62      0.68     33373
    nh_black       0.67      0.32      0.43     28535
    nh_white       0.80      0.94      0.87    134206

    accuracy                           0.79    200000
   macro avg       0.73      0.55      0.60    200000
weighted avg       0.77      0.79      0.76    200000

[[  1188    471    361   1866]
 [    82  20775    955  11561]
 [   185   1760   9152  17438]
 [   251   4698   3235 126022]]


## Save model

In [11]:
model.save('../dataverse_files/fl_voter_reg/lstm/fl_all_ln_lstm.h5')

In [12]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_all_ln_vocab.csv', index=False, encoding='utf-8')

# Uncertainity Model

In [13]:
print('Build uncertainty model...')

#model_uncrtn = Sequential()
input_ = keras.layers.Input(shape=(feature_len))
layer_ = Embedding(num_words, 32) (input_)
layer_ = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(layer_)
layer_ = Dropout(0.25)(layer_, training=True)
output = Dense(num_classes, activation='softmax') (layer_)

model_uncrtn = keras.models.Model(inputs=input_, outputs=output)

# try using different optimizers and different optimizer configs
model_uncrtn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model_uncrtn.summary())

Build uncertainty model...
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 32)            35296     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 118,244
Trainable params: 118,244
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
print('Train...')
model_uncrtn.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model_uncrtn.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15
22500/22500 [==============================] - 397s 18ms/step - loss: 0.7511 - accuracy: 0.7345 - val_loss: 0.6844 - val_accuracy: 0.7626
Epoch 2/15
22500/22500 [==============================] - 391s 17ms/step - loss: 0.6783 - accuracy: 0.7644 - val_loss: 0.6601 - val_accuracy: 0.7710
Epoch 3/15
22500/22500 [==============================] - 393s 17ms/step - loss: 0.6616 - accuracy: 0.7704 - val_loss: 0.6505 - val_accuracy: 0.7753
Epoch 4/15
22500/22500 [==============================] - 391s 17ms/step - loss: 0.6525 - accuracy: 0.7739 - val_loss: 0.6443 - val_accuracy: 0.7778
Epoch 5/15
22500/22500 [==============================] - 391s 17ms/step - loss: 0.6472 - accuracy: 0.7764 - val_loss: 0.6414 - val_accuracy: 0.7798
Epoch 6/15
22500/22500 [==============================] - 392s 17ms/step - loss: 0.6423 - accuracy: 0.7773 - val_loss: 0.6380 - val_accuracy: 0.7812
Epoch 7/15
22500/22500 [==============================] - 391s 17ms/step - loss: 0.6396 - accurac

In [15]:
predictions = []

ITER=100

for _ in range(ITER):
    predictions.append(model_uncrtn.predict(X_test, verbose=1))

6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 22s
6250/6250 - 22s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/6250 - 21s
6250/625

In [16]:
predict_array = np.array(predictions)

In [33]:
# Creating a seperate array for each measurement
mean_arr = predict_array.mean(axis=0).reshape(-1,4)
std_arr = predict_array.std(axis=0).reshape(-1,4)
pct_5_arr = np.quantile(predict_array, .05, axis=0).reshape(-1,4)
pct_95_arr = np.quantile(predict_array, .95, axis=0).reshape(-1,4)

In [34]:
# Selecting the class based on teh mean value with the highest probability
final_pred_arr = mean_arr.argmax(axis=1)

In [35]:
final_pred_arr.shape

(200000,)

# Creating Final DataFrame

In [42]:
predict_df = pd.DataFrame(columns = ['pred', 'pred_5', 'pred_95', 'pred_se'])

for i in range(len(final_pred_arr)):
    pred = final_pred_arr[i]
    pred_5 = pct_5_arr[i,pred]
    pred_95 = pct_95_arr[i,pred]
    pred_se = std_arr[i,pred]
    
    predict_df = predict_df.append({'pred':pred, 
                                    'pred_5':pred_5, 
                                    'pred_95': pred_95, 
                                    'pred_se':pred_se}, ignore_index=True)

In [43]:
predict_df

,pred,pred_5,pred_95,pred_se
0,3.0,0.711988,0.805355,0.029818
1,3.0,0.786448,0.867701,0.023872
2,1.0,0.286286,0.520776,0.069999
3,3.0,0.791124,0.854948,0.018592
4,3.0,0.760636,0.848683,0.026504
...,...,...,...,...
199995,3.0,0.699482,0.766014,0.019765
199996,2.0,0.888805,0.978085,0.026221
199997,3.0,0.841669,0.907399,0.020203
199998,3.0,0.865521,0.922698,0.018780
